## Calculate NPMRDS level of travel-time reliability (LOTTR) metric for all vehicles

### Import required packages

In [ ]:
import pandas as pd
import datetime
from dbfread import DBF

### Methodology
The methodology for calculating the level of travel-time reliabililty (LOTTR) metric for all vehicles from NPMRDS data is documented in the FHWA document  
[National Performance Measures for Congestions, Reliability, and Freight, and CMAQ Traffic Congestion](https://www.fhwa.dot.gov/tpm/guidance/hif18040.pdf).

In summary, we:
1. Calculate the LOTTR metric for each TMC for each of the four National Highway Performance (time) Periods for all vehicles, and then
2. Caclulate the single MPO region-wide LOTTR metric from \(1\).

### Required input 1: speed and travel-time data
This notebook uses travel-time data downloaded from the NPMRDS Analytics Probe Data Analytics Portal in [RITIS](https://ritis.org).  

The specification of the data to download from RITS is as follows:
* All NPMRDS TMCs in the Boston Region MPO area. This list of TMCs is found in the file __brmpo\_npmrds\_tmcs\_2019.txt__.
* January 1, 2019 through December 31, 2019, from 6:00 AM to 8:00 PM
* With data aggregated in 15 minute units

The user needs to have downloaded this data from RITIS and extracted the CSV file containing speed and travel-time data from teh ZIP archive delivered by the npmrds-analytics email system.  

The location of this CSV file is specified by the variable __ritis\_data\_csv__.

In [ ]:
ritis_data_csv =   r'//lilliput/bkrepp/jupyter_notebooks/npmrds-tt-reliability/data/npmrds-all-vehicles-2019.csv'

In [ ]:
ritis_df = pd.read_csv(ritis_data_csv)

In [ ]:
ritis_df.head(10)

### Required input 2: DBF file containing attribute table for NPMRDS shapefile

In [ ]:
# DBF file with attribute table for NPMPRDS shapefile
tmc_attr_table_fn = r'//lilliput/bkrepp/jupyter_notebooks/npmrds-tt-reliability/shapefile/BRMPO_NPMRDS_TMCS_2019.dbf'

In [ ]:
tmc_attr_table_dbf = DBF(tmc_attr_table_fn)
tmc_attr_table_df = pd.DataFrame(iter(tmc_attr_table_dbf))

In [ ]:
tmc_attr_table_df.head(10)

#### The 4 National Highway Performance Program (NHPP) time periods for "all vehicles"
* Weekdays 6 AM to 10 AM
* Weekdays 10 AM to 4 PM
* Weedkays 4 PM to 8 PM
* Weekends 6 AM to 8 PM

In [ ]:
# Enums (symbolic constants) for NHPP_Periods:
NHPP_NONE         =  0
NHPP_WEEKDAY_6_10 =  1
NHPP_WEEKDAY_10_4 =  2
NHPP_WEEKDAY_4_8  =  3
NHPP_WEEKEND_6_8  =  4

### Utility function to map a given day-of-week and hour to a NHPP time period.

In [ ]:
# Utility function to map a given day-of-week (dow) and hour to a NHPP time period.
# NOTES:
#     1. In the input data, 'hours' are given using the 24-hour (NOT 12-hour) clock.
#        For example, 4 PM == 16 hours
#     2. The parametr 'dow' is expressed in terms of the Python datetime.date.weekday() function,
#        i.e., 0 -> Monday, 1 -> Tuesday, ..., 5 -> Saturday, 6 -> Sunday
#
def get_nhpp_period(dow, hour):
	retval = NHPP_NONE 
	if (dow == 5 or dow == 6):
		if hour >=6 and hour < 20:
			retval = NHPP_WEEKEND_6_8
		else:
			retval = NHPP_NONE
	elif hour >=6 and hour < 10:
		retval = NHPP_WEEKDAY_6_10
	elif hour >= 10 and hour < 16:
		retval = NHPP_WEEKDAY_10_4
	elif hour >= 16 and hour < 20:
		retval = NHPP_WEEKDAY_4_8
	else:
		retval = NHPP_NONE
	# endif
	return retval
# end_def get_nhpp_period()

### 'Explode' the _measurement_timestamp_ field of the _ritis\_df_

In [ ]:
ritis_df['datepart'] = ritis_df.apply(lambda row : row['measurement_tstamp'].split(' ')[0], axis=1)

In [ ]:
ritis_df['timepart'] = ritis_df.apply(lambda row : row['measurement_tstamp'].split(' ')[1], axis=1)

In [ ]:
ritis_df['mo'] = ritis_df.apply(lambda row : int(row['datepart'].split('-')[1]), axis=1)

In [ ]:
ritis_df['dy'] = ritis_df.apply(lambda row : int(row['datepart'].split('-')[2]), axis=1)

In [ ]:
ritis_df.head(10)

In [ ]:
# Assumes all records are for 2019
ritis_df['dow'] = ritis_df.apply(lambda row : datetime.date(2019, row['mo'], row['dy']).weekday(), axis=1)

In [ ]:
ritis_df['hour'] = ritis_df.apply(lambda row: int(row['timepart'].split(':')[0]), axis=1)

In [ ]:
ritis_df['nhpp_period'] = ritis_df.apply(lambda row : get_nhpp_period(row['dow'], row['hour']), axis=1)

### Calculate LOTTR metric for all TMCs for  NHPP time period 1

In [ ]:
period_1_df = ritis_df.copy(deep=True)
period_1_df = period_1_df[period_1_df['nhpp_period'] == NHPP_WEEKDAY_6_10]

g50 = period_1_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_1_g50_df = g50.to_frame()
period_1_g50_df = period_1_g50_df.rename(columns={'travel_time_seconds' : "p1_tt_secs_50pct"})

g80 = period_1_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.8)
period_1_g80_df = g80.to_frame()
period_1_g80_df = period_1_g80_df.rename(columns={'travel_time_seconds' : "p1_tt_secs_80pct"})

period_1_stats_df = period_1_g50_df.merge(right=period_1_g80_df, left_on='tmc_code', right_on='tmc_code')
period_1_stats_df['p1_lottr'] = round(period_1_stats_df['p1_tt_secs_80pct'] / period_1_stats_df['p1_tt_secs_50pct'], 2)

In [ ]:
period_1_stats_df.head(10)

### Calculate LOTTR metric for all TMCs for NHPP time period 2

In [ ]:
period_2_df = ritis_df.copy(deep=True)
period_2_df = period_2_df[period_2_df['nhpp_period'] == NHPP_WEEKDAY_10_4]

g50 = period_2_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_2_g50_df = g50.to_frame()
period_2_g50_df = period_2_g50_df.rename(columns={'travel_time_seconds' : "p2_tt_secs_50pct"})

g80 = period_2_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.8)
period_2_g80_df = g80.to_frame()
period_2_g80_df = period_2_g80_df.rename(columns={'travel_time_seconds' : "p2_tt_secs_80pct"})

period_2_stats_df = period_2_g50_df.merge(right=period_2_g80_df, left_on='tmc_code', right_on='tmc_code')
period_2_stats_df['p2_lottr'] = round(period_2_stats_df['p2_tt_secs_80pct'] / period_2_stats_df['p2_tt_secs_50pct'], 2)

In [ ]:
period_2_stats_df.head(10)

### Calculate LOTTR metric for all TMCs for NHPP time period 3

In [ ]:
period_3_df = ritis_df.copy(deep=True)
period_3_df = period_3_df[period_3_df['nhpp_period'] == NHPP_WEEKDAY_4_8 ]

g50 = period_3_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_3_g50_df = g50.to_frame()
period_3_g50_df = period_3_g50_df.rename(columns={'travel_time_seconds' : "p3_tt_secs_50pct"})

g80 = period_3_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.8)
period_3_g80_df = g80.to_frame()
period_3_g80_df = period_3_g80_df.rename(columns={'travel_time_seconds' : "p3_tt_secs_80pct"})

period_3_stats_df = period_3_g50_df.merge(right=period_3_g80_df, left_on='tmc_code', right_on='tmc_code')
period_3_stats_df['p3_lottr'] = round(period_3_stats_df['p3_tt_secs_80pct'] / period_3_stats_df['p3_tt_secs_50pct'], 2)

In [ ]:
period_3_stats_df.head(10)

### Calculate LOTTR metric for all TMCs for NHPP time period 4

In [ ]:
period_4_df = ritis_df.copy(deep=True)
period_4_df = period_4_df[period_4_df['nhpp_period'] == NHPP_WEEKEND_6_8]

g50 = period_4_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.5)
period_4_g50_df = g50.to_frame()
period_4_g50_df = period_4_g50_df.rename(columns={'travel_time_seconds' : "p4_tt_secs_50pct"})

g80 = period_4_df.groupby('tmc_code')['travel_time_seconds'].quantile(q=0.8)
period_4_g80_df = g80.to_frame()
period_4_g80_df = period_4_g80_df.rename(columns={'travel_time_seconds' : "p4_tt_secs_80pct"})

period_4_stats_df = period_4_g50_df.merge(right=period_4_g80_df, left_on='tmc_code', right_on='tmc_code')
period_4_stats_df['p4_lottr'] = round(period_4_stats_df['p4_tt_secs_80pct'] / period_4_stats_df['p4_tt_secs_50pct'], 2)

In [ ]:
period_4_stats_df.head(10)

### Join time-period specific dataframes into a single dataframe

In [ ]:
j1 = period_1_stats_df.merge(right=period_2_stats_df, left_on='tmc_code', right_on='tmc_code')
j2 = j1.merge(right=period_3_stats_df, left_on='tmc_code', right_on='tmc_code')
j3 = j2.merge(right=period_4_stats_df, left_on='tmc_code', right_on='tmc_code')

In [ ]:
j3 = j3.drop(columns=['p1_tt_secs_50pct', 'p1_tt_secs_80pct', 'p2_tt_secs_50pct', 'p2_tt_secs_80pct',
                      'p3_tt_secs_50pct', 'p3_tt_secs_80pct', 'p4_tt_secs_50pct', 'p4_tt_secs_80pct'])

In [ ]:
j3.head(10)

### Prepare final dataframe with LOTTR metric for each TMC for output

In [ ]:
final_df = j3.rename(columns={'p1_lottr' : 'LOTTR Weekday 6 AM -10 AM',
                              'p2_lottr' : 'LOTTR Weekday 10 AM - 4 PM',
                              'p3_lottr' : 'LOTTR Weekday 4 PM - 8 PM',
                              'p4_lottr' : 'LOTTR Weekend 6 AM - 8 PM'})

In [ ]:
final_df.head(10)

### Export dataframe with LOTTR metric for each TMC to a CSV file

In [ ]:
output_csv_fn = r'//lilliput/bkrepp/jupyter_notebooks/npmrds-tt-reliability/lottr_all_vehicles.csv'

In [ ]:
final_df.to_csv(output_csv_fn)

### Calculate the single regional LOTTR metric

This metric is given by the formula:
$$
{NHPP\_Reliability\_Measure} = \frac{\sum \limits _{r=1} ^{R} {SL}_{i} * {AV}_{i} * {OF}_{j}} {\sum \limits _{t=1} ^{T} {SL}_{i} * {AV}_{i} * {OF}_{j}}
$$

Where:  
${SL}_{i}$ = the segment length of the reporting segment _i_ \(nearest thousandth\) for the highway system considered.  
$ {AV}_{i}$ = annual traffic volume of reporting segment _i_. \(See below.\)  
$ {OF}_{j}$ = occupancy factor for vehicles on the NHS with a specific geographic area _j_ within the State / Metropolitan planning area. \(This value is 1.7 for Massachusetts.\)  
__R__ = total number of Interstate and non-Interstate reporting segments exhibiting an LOTTR below 1.50 for all 4 time periods.  
__T__ = total number of Interstate and non-Interstate reporting segments.

And,  
$ {AV}_{j}$ = AADT * Directional_Factor * 365  

Where:
Directional_Factor is 0.5 if the _faciltype_ of the TMC is 2 or 6, and is 1 if the _faciltype_ of the TMC is 1. 

In [ ]:
# Prep dataframe to be joined to TMC attribute table data. 
# Add column for 'average volume'
j3['AV'] = 0

In [ ]:
# Join DF with TMC-level stats with TMC attribute table
T_df = j3.merge(right=tmc_attr_table_df, left_on='tmc_code', right_on='tmc')

In [ ]:
# Calculate AV value
T_df['AV'] = T_df.apply(lambda row: row['aadt'] * (0.5 if row['faciltype'] == 2 else 1.0) *365, axis=1)

In [ ]:
# FHWA occupancy factor for 'all vehicles'
T_df['OF'] = 1.7

In [ ]:
# Calculate term used in numerator and denominator of NHPP reliability measure
# Round 'miles' to nearest thousandth, per FHWA doc
T_df['lottr_measure'] = round(T_df['miles'], 3) * T_df['AV']  * T_df['OF']

In [ ]:
# Calculate denominator: sum of 'lottr_measure' for all reporting TMCs
denominator = T_df['lottr_measure'].sum()
denominator

In [ ]:
# Prep for calculating numerator: get DF of TMCs with LOTTR < 1.5 for all 4 time periods
R_df = T_df[(T_df['p1_lottr'] < 1.5) & (T_df['p2_lottr'] < 1.5) & (T_df['p3_lottr'] < 1.5) & (T_df['p4_lottr'] < 1.5)]

In [ ]:
numerator = R_df['lottr_measure'].sum()
numerator

In [ ]:
NHPP_Reliability_Measure = numerator / denominator
NHPP_Reliability_Measure